In [1]:
from kaggle_environments.envs.halite.helpers import *
from kaggle_environments import make
import kaggle_environments
import random
import pickle

import numpy as np
from time import time
import torch
from collections import deque

from obs_parser_v2 import Obs, actionToTensor, logitToAction, generate_offset_map

from ship_agent import ActAgent

In [2]:
sl_agent = ActAgent(cin = 9, vec_size = 21, out_ship=6, out_shipyard=2, random_seed=2)

In [3]:
def print_dict(d):
    print('\t'.join([f'{key}: {val:.6f}' for key, val in d.items()]))

In [4]:
def string_dict(d):
    return '\t'.join([f'{key}: {val:.6f}' for key, val in d.items()])

In [5]:
from replayDownload import *
import json
import os

In [6]:
sl_agent.load_pretrained_weight('fei_stage1_v6/checkpoint_shipactor_17_36720.pth', 'mzo_refine3/checkpoint_shipyardactor_mzo_32_10380.pth')

In [ ]:
#submissionId = 16425049 # Robiland 16680981
#submissionId = 16454608 # mzo 16647790
#submissionIds = [16720545, 16706186] # TeamVQ new
#submissionId = 16302889 # TeamVQ
#submissionId = 16466155 ## Tom
#mzo [16942343, 16942830]
#submissionIds = [17011711, 17041870, 16994015, 16842830, 16962227, 16985357] # mzo
#submissionIds = [17020115, 17007878, 16961133, 17028760, 17027337, 16680981] # Robiland
#submissionIds = [16662467, 16668218] #fei
submissionIds = [17092277, 17075747, 17068749, 17067323, 17064756, 17056842, 17047360, 17044587, 17035283, 17034794, 17004948, 17003453, 17002938, 17000660, \
                 16996120, 16986553, 16985940, 16930073, 16801544, 16801309] #fei
#submissionIds = [17035283, 17003453, 17000660, 16996120] #fei_new
#allEpisodes = list_episodes_for_submission(submissionId)
#submissionIds = [16727959] # RL Theo Cachet
allEpisodes = [list_episodes_for_submission(subid) for subid in submissionIds]

i_episode = 0
save_folder = 'fei_stage1_v11'
with open(save_folder + '/log.dat', 'w') as f:
    for epoch in range(40):
        for subid in submissionIds:
            allEpisodes = list_episodes_for_submission(subid)
            epiodeList = allEpisodes['result']['episodes']
            random.shuffle(epiodeList)
            for episode in epiodeList[::-1]:
                t0 = time()
                i_episode += 1
                replayId = episode['id']
                agents = episode['agents']
                agentsId = [d['submissionId'] for d in agents]
                #print(agentsId)
                
                if os.path.exists(f'fei_replay/{replayId}.pkl'):
                    with open(f'fei_replay/{replayId}.pkl', 'rb') as p:
                        replay = pickle.load(p)
                else:
                    try:
                        replay = get_episode_replay(replayId)
                        with open(f'fei_replay/{replayId}.pkl', 'wb') as p:
                            pickle.dump(replay, p)
                    except:
                        print('failed download replay')
                        continue
                try:
                    replay_dict = json.loads(replay['result']['replay'])
                    for i, v in enumerate(replay_dict['rewards']):
                        if v == None:
                            replay_dict['rewards'][i] = -2000
                    #winnerId = replay_dict['rewards'].index(max(replay_dict['rewards']))
                    winnerId = agentsId.index(subid)
                    scores = replay_dict['rewards']
                    player_score = scores[winnerId]
                    scores.sort()
                    if player_score < scores[-2]:
                        continue
                    #print(f"winner: {winnerId},\treward: {replay_dict['rewards']}")
                    steps = len(replay_dict['steps'])
                    if steps < 100:
                        continue
                    #steps = min(steps, 75)
                    for step in range(84):
                        observation = replay_dict['steps'][step][0]['observation']
                        if winnerId != 0:
                            observation['player'] = winnerId

                        my_action = replay_dict['steps'][step+1][winnerId]['action']
                        obs = Obs(observation)
                        if len(obs.shipPosDict) >= 1:
                            repeat = max(1, 20 // len(obs.shipPosDict))
                            #repeat = 1
                            for _ in range(repeat):
                                state = obs.state
                                state1, state2 = state
                                action = actionToTensor(my_action, obs.shipPosDict, obs.shipyardPosDict)

                                for key, val in obs.shipPosDict.items():
                                    r, c = key
                                    c_state1 = generate_offset_map(state1, r, c)
                                    target = np.argmax(action[:6, r, c])
                                    #print(f'ship state1: {c_state1}')
                                    #print(f'ship target: {target}')
                                    sl_agent.add(c_state1, state2, target, True)
                                for key, val in obs.shipyardPosDict.items():
                                    r, c = key
                                    c_state1 = generate_offset_map(state1, r, c)
                                    target = np.argmax(action[6:, r, c])
                                    sl_agent.add(c_state1, state2, target, False)

                                """### rot90
                                state1rt = np.rot90(state1, k=1, axes=(1,2))
                                actionrt = np.rot90(action, k=1, axes=(1,2))
                                actionrt[2], actionrt[3], actionrt[1], actionrt[0] = actionrt[0].copy(), actionrt[1].copy(), \
                                                                                     actionrt[2].copy(), actionrt[3].copy()

                                for key, val in obs.shipPosDict.items():
                                    r, c = key
                                    r, c = 20-c, r
                                    c_state1 = generate_offset_map(state1rt, r, c)
                                    target = np.argmax(actionrt[:6, r, c])
                                    #print(f'ship state1: {c_state1}')
                                    #print(f'ship target: {target}')
                                    sl_agent.add(c_state1, state2, target, True)
                                for key, val in obs.shipyardPosDict.items():
                                    r, c = key
                                    r, c = 20-c, r
                                    c_state1 = generate_offset_map(state1rt, r, c)
                                    target = np.argmax(actionrt[6:, r, c])
                                    sl_agent.add(c_state1, state2, target, False)"""

                                ### LR flip
                                state1lr = state1[:,:,::-1].copy()
                                actionlr = action[:,:,::-1].copy()
                                actionlr[2,:,:], actionlr[3,:,:] = actionlr[3,:,:].copy(), actionlr[2,:,:].copy()
                                for key, val in obs.shipPosDict.items():
                                    r, c = key
                                    c = 20 - c
                                    c_state1 = generate_offset_map(state1lr, r, c)
                                    target = np.argmax(actionlr[:6, r, c])
                                    #print(f'ship state1: {c_state1}')
                                    #print(f'ship target: {target}')
                                    sl_agent.add(c_state1, state2, target, True)
                                for key, val in obs.shipyardPosDict.items():
                                    r, c = key
                                    c = 20 - c
                                    c_state1 = generate_offset_map(state1lr, r, c)
                                    target = np.argmax(actionlr[6:, r, c])
                                    sl_agent.add(c_state1, state2, target, False)

                                """### LR flip + rot90
                                state1lrrt = np.rot90(state1lr, k=1, axes=(1,2))
                                actionlrrt = np.rot90(actionlr, k=1, axes=(1,2))
                                actionlrrt[2], actionlrrt[3], actionlrrt[1], actionlrrt[0] = actionlrrt[0].copy(), actionlrrt[1].copy(), \
                                                                                             actionlrrt[2].copy(), actionlrrt[3].copy()

                                for key, val in obs.shipPosDict.items():
                                    r, c = key
                                    c = 20 - c
                                    r, c = 20-c, r
                                    c_state1 = generate_offset_map(state1lrrt, r, c)
                                    target = np.argmax(actionlrrt[:6, r, c])
                                    #print(f'ship state1: {c_state1}')
                                    #print(f'ship target: {target}')
                                    sl_agent.add(c_state1, state2, target, True)
                                for key, val in obs.shipyardPosDict.items():
                                    r, c = key
                                    c = 20 - c
                                    r, c = 20-c, r
                                    c_state1 = generate_offset_map(state1lrrt, r, c)
                                    target = np.argmax(actionlrrt[6:, r, c])
                                    sl_agent.add(c_state1, state2, target, False)"""


                                ### UD flip
                                state1ud = state1[:,::-1,:].copy()
                                actionud = action[:,::-1,:].copy()
                                actionud[0,:,:], actionud[1,:,:] = actionud[1,:,:].copy(), actionud[0,:,:].copy()
                                for key, val in obs.shipPosDict.items():
                                    r, c = key
                                    r = 20 - r
                                    c_state1 = generate_offset_map(state1ud, r, c)
                                    target = np.argmax(actionud[:6, r, c])
                                    #print(f'ship target: {target}')
                                    sl_agent.add(c_state1, state2, target, True)
                                for key, val in obs.shipyardPosDict.items():
                                    r, c = key
                                    r = 20 - r
                                    c_state1 = generate_offset_map(state1ud, r, c)
                                    target = np.argmax(actionud[6:, r, c])
                                    sl_agent.add(c_state1, state2, target, False)

                                """### UP flip + rot90
                                state1udrt = np.rot90(state1ud, k=1, axes=(1,2))
                                actionudrt = np.rot90(actionud, k=1, axes=(1,2))
                                actionudrt[2], actionudrt[3], actionudrt[1], actionudrt[0] = actionudrt[0].copy(), actionudrt[1].copy(), \
                                                                                             actionudrt[2].copy(), actionudrt[3].copy()

                                for key, val in obs.shipPosDict.items():
                                    r, c = key
                                    r = 20 - r
                                    r, c = 20-c, r
                                    c_state1 = generate_offset_map(state1udrt, r, c)
                                    target = np.argmax(actionudrt[:6, r, c])
                                    #print(f'ship state1: {c_state1}')
                                    #print(f'ship target: {target}')
                                    sl_agent.add(c_state1, state2, target, True)
                                for key, val in obs.shipyardPosDict.items():
                                    r, c = key
                                    r = 20 - r
                                    r, c = 20-c, r
                                    c_state1 = generate_offset_map(state1udrt, r, c)
                                    target = np.argmax(actionudrt[6:, r, c])
                                    sl_agent.add(c_state1, state2, target, False)"""



                                ### LRUD flip
                                state1lrud = state1lr[:,::-1,:].copy()
                                actionlrud = actionlr[:,::-1,:].copy()
                                actionlrud[0,:,:], actionlrud[1,:,:] = actionlrud[1,:,:].copy(), actionlrud[0,:,:].copy()
                                for key, val in obs.shipPosDict.items():
                                    r, c = key
                                    r, c = 20 - r, 20 -c
                                    c_state1 = generate_offset_map(state1lrud, r, c)
                                    target = np.argmax(actionlrud[:6, r, c])
                                    #print(f'ship target: {target}')
                                    sl_agent.add(c_state1, state2, target, True)
                                for key, val in obs.shipyardPosDict.items():
                                    r, c = key
                                    r, c = 20 - r, 20 -c
                                    c_state1 = generate_offset_map(state1lrud, r, c)
                                    target = np.argmax(actionlrud[6:, r, c])
                                    sl_agent.add(c_state1, state2, target, False)

                                """### LRUD flip + rot90
                                state1lrudrt = np.rot90(state1lrud, k=1, axes=(1,2))
                                actionlrudrt = np.rot90(actionlrud, k=1, axes=(1,2))
                                actionlrudrt[2], actionlrudrt[3], actionlrudrt[1], actionlrudrt[0] = actionlrudrt[0].copy(), actionlrudrt[1].copy(), \
                                                                                                     actionlrudrt[2].copy(), actionlrudrt[3].copy()

                                for key, val in obs.shipPosDict.items():
                                    r, c = key
                                    r, c = 20 - r, 20 -c
                                    r, c = 20-c, r
                                    c_state1 = generate_offset_map(state1lrudrt, r, c)
                                    target = np.argmax(actionlrudrt[:6, r, c])
                                    #print(f'ship state1: {c_state1}')
                                    #print(f'ship target: {target}')
                                    sl_agent.add(c_state1, state2, target, True)
                                for key, val in obs.shipyardPosDict.items():
                                    r, c = key
                                    r, c = 20 - r, 20 -c
                                    r, c = 20-c, r
                                    c_state1 = generate_offset_map(state1lrudrt, r, c)
                                    target = np.argmax(actionlrudrt[6:, r, c])
                                    sl_agent.add(c_state1, state2, target, False)"""

                    shipmetric, shipyardmetric = sl_agent.step()
                    #print(f'Episode No. {i_episode}')
                    print_dict(shipmetric)
                    #print_dict(shipyardmetric)

                    if i_episode % 30 == 0:
                        print('==============================================================================')
                        print('\rEpoch {}\tEpisode {}\tCost Time {:.2f}\t'.format(epoch, i_episode, time()-t0))
                        f.write('\rEpoch {}\tEpisode {}\tCost Time {:.2f}\t'.format(epoch, i_episode, time()-t0))
                        f.write(string_dict(shipmetric) + '\n')
                        f.flush()
                        torch.save(sl_agent.actorship.state_dict(), f'{save_folder}/checkpoint_shipactor_{epoch}_{i_episode}.pth')
                        #torch.save(sl_agent.actorshipyard.state_dict(), f'mzo_refine6/checkpoint_shipyardactor_mzo_{epoch}_{i_episode}.pth')
                except:
                    pass
        sl_agent.lr_step()



















loss: 0.430679	acc: 0.592969
loss: 0.387855	acc: 0.593281
loss: 0.410290	acc: 0.608906
loss: 0.459869	acc: 0.616563
loss: 0.350262	acc: 0.595781
loss: 0.435201	acc: 0.609375
loss: 0.429685	acc: 0.619375
loss: 0.334478	acc: 0.608437
loss: 0.310228	acc: 0.594063
loss: 0.374462	acc: 0.609688
loss: 0.373828	acc: 0.598281
loss: 0.386988	acc: 0.614688
loss: 0.346763	acc: 0.617500
loss: 0.509039	acc: 0.606250
loss: 0.386452	acc: 0.614844
loss: 0.389082	acc: 0.607187
loss: 0.293606	acc: 0.606406
loss: 0.443620	acc: 0.612812
loss: 0.465771	acc: 0.605313
loss: 0.343830	acc: 0.626250
loss: 0.383401	acc: 0.615625
loss: 0.281766	acc: 0.620000
loss: 0.396194	acc: 0.617500
loss: 0.300192	acc: 0.613281
loss: 0.333889	acc: 0.619531
Epoch 0	Episode 60	Cost Time 1.53	
loss: 0.394539	acc: 0.617500
loss: 0.372523	acc: 0.610000
loss: 0.332338	acc: 0.615781
loss: 0.403171	acc: 0.620000
loss: 0.345629	acc: 0.608594
loss: 0.291949	acc: 0.592187
loss: 0.390140	acc: 0.615156
loss: 0.306208	acc:

loss: 0.368432	acc: 0.612812
loss: 0.337237	acc: 0.631563
loss: 0.451564	acc: 0.617812
loss: 0.420450	acc: 0.625469
loss: 0.353219	acc: 0.619375
loss: 0.334228	acc: 0.620781
loss: 0.376658	acc: 0.621406
loss: 0.346659	acc: 0.626719
loss: 0.287982	acc: 0.625156
loss: 0.365427	acc: 0.633437
loss: 0.403669	acc: 0.631719
Epoch 0	Episode 390	Cost Time 1.93	
loss: 0.253633	acc: 0.621094
loss: 0.340561	acc: 0.623750
loss: 0.324844	acc: 0.636719
loss: 0.346197	acc: 0.625000
loss: 0.269383	acc: 0.622969
loss: 0.338743	acc: 0.626719
loss: 0.337347	acc: 0.621875
loss: 0.336244	acc: 0.620000
loss: 0.322024	acc: 0.624219
loss: 0.362289	acc: 0.629531
loss: 0.373653	acc: 0.627188
loss: 0.343097	acc: 0.625000
loss: 0.382329	acc: 0.626406
loss: 0.378690	acc: 0.624375
loss: 0.269510	acc: 0.630469
loss: 0.396438	acc: 0.621406
loss: 0.292736	acc: 0.623437
loss: 0.303218	acc: 0.623125
loss: 0.354846	acc: 0.633750
loss: 0.331494	acc: 0.630625
loss: 0.262074	acc: 0.631719
loss: 0.314709	acc: 0.632031
loss: 0

loss: 0.356767	acc: 0.612656
loss: 0.376471	acc: 0.608281
loss: 0.326649	acc: 0.601875
loss: 0.275832	acc: 0.611875
loss: 0.303947	acc: 0.601719
loss: 0.410062	acc: 0.594219
loss: 0.371218	acc: 0.605000
loss: 0.348229	acc: 0.600781
loss: 0.363981	acc: 0.595469
loss: 0.274890	acc: 0.613750
loss: 0.383788	acc: 0.603437
Epoch 0	Episode 750	Cost Time 1.97	
loss: 0.406929	acc: 0.603750
loss: 0.402092	acc: 0.613750
loss: 0.317940	acc: 0.600625
loss: 0.385465	acc: 0.604844
loss: 0.329088	acc: 0.604844
loss: 0.323623	acc: 0.600625
loss: 0.385727	acc: 0.605000
loss: 0.268545	acc: 0.603437
loss: 0.316091	acc: 0.599844
loss: 0.377352	acc: 0.607969
loss: 0.347920	acc: 0.597656
loss: 0.358680	acc: 0.609688
loss: 0.301983	acc: 0.607656
loss: 0.383867	acc: 0.615938
loss: 0.270614	acc: 0.608281
loss: 0.412119	acc: 0.601719
loss: 0.446719	acc: 0.603594
loss: 0.323815	acc: 0.609844
loss: 0.483666	acc: 0.595313
Epoch 0	Episode 780	Cost Time 1.99	
loss: 0.347995	acc: 0.591562
loss: 0.389580	acc: 0.608437


loss: 0.250931	acc: 0.601250
loss: 0.345531	acc: 0.602031
loss: 0.333224	acc: 0.614688
loss: 0.390177	acc: 0.607031
Epoch 0	Episode 1110	Cost Time 1.97	
loss: 0.370381	acc: 0.605313
loss: 0.302217	acc: 0.612812
loss: 0.328973	acc: 0.616094
loss: 0.346926	acc: 0.612031
loss: 0.296393	acc: 0.614062
loss: 0.360860	acc: 0.607656
loss: 0.333222	acc: 0.605625
loss: 0.309891	acc: 0.618125
loss: 0.355594	acc: 0.604531
loss: 0.442631	acc: 0.608906
loss: 0.391696	acc: 0.610781
loss: 0.297522	acc: 0.612500
loss: 0.336607	acc: 0.607969
loss: 0.350562	acc: 0.613437
loss: 0.260305	acc: 0.616406
loss: 0.317356	acc: 0.618281
loss: 0.333490	acc: 0.603750
loss: 0.293618	acc: 0.616719
loss: 0.317554	acc: 0.605781
loss: 0.329364	acc: 0.605313
loss: 0.295609	acc: 0.609219
loss: 0.393420	acc: 0.597031
loss: 0.399998	acc: 0.602969
loss: 0.303070	acc: 0.603594
loss: 0.401660	acc: 0.603125
loss: 0.331784	acc: 0.620625
loss: 0.355312	acc: 0.613281
loss: 0.355242	acc: 0.602812
loss: 0.313133	acc: 0.607969
loss: 

loss: 0.280506	acc: 0.617031
loss: 0.337220	acc: 0.621563
loss: 0.419074	acc: 0.617969
loss: 0.393007	acc: 0.620781
loss: 0.428354	acc: 0.617969
loss: 0.375983	acc: 0.611406
loss: 0.275718	acc: 0.618750
loss: 0.310879	acc: 0.617812
loss: 0.399733	acc: 0.620000
loss: 0.357677	acc: 0.619219
loss: 0.322316	acc: 0.631719
loss: 0.358184	acc: 0.628281
loss: 0.408181	acc: 0.618594
loss: 0.410492	acc: 0.622656
loss: 0.330541	acc: 0.626406
loss: 0.251024	acc: 0.619844
loss: 0.353403	acc: 0.621875
loss: 0.361814	acc: 0.618281
loss: 0.279023	acc: 0.624687
loss: 0.344808	acc: 0.616250
loss: 0.428371	acc: 0.615781
loss: 0.356267	acc: 0.614844
loss: 0.275087	acc: 0.617031
loss: 0.307659	acc: 0.616875
loss: 0.364667	acc: 0.614062
loss: 0.393001	acc: 0.614219
loss: 0.296168	acc: 0.630156
loss: 0.309403	acc: 0.627656
loss: 0.313185	acc: 0.623281
loss: 0.354748	acc: 0.630469
loss: 0.425673	acc: 0.616094
loss: 0.272935	acc: 0.627500
loss: 0.315335	acc: 0.620469
loss: 0.206484	acc: 0.623281
loss: 0.267193

...

loss: 0.407419	acc: 0.610938
loss: 0.383596	acc: 0.617656
loss: 0.350151	acc: 0.612500
loss: 0.333530	acc: 0.614844
loss: 0.261930	acc: 0.614062
loss: 0.357875	acc: 0.613125
loss: 0.244681	acc: 0.613437
loss: 0.350594	acc: 0.601250
loss: 0.318068	acc: 0.612500
loss: 0.404363	acc: 0.613594
loss: 0.361154	acc: 0.625156
loss: 0.332814	acc: 0.618437
loss: 0.303685	acc: 0.610313
loss: 0.339548	acc: 0.615469
loss: 0.317279	acc: 0.625469
loss: 0.426033	acc: 0.623750
loss: 0.358167	acc: 0.610938
loss: 0.362000	acc: 0.619844
loss: 0.312920	acc: 0.622812
loss: 0.411999	acc: 0.616094
loss: 0.339906	acc: 0.613906
Epoch 11	Episode 33240	Cost Time 2.11	
loss: 0.365144	acc: 0.622031
loss: 0.333819	acc: 0.610000
loss: 0.367730	acc: 0.612812
loss: 0.352917	acc: 0.614688
loss: 0.334113	acc: 0.624844
loss: 0.322579	acc: 0.625469
loss: 0.319651	acc: 0.620625
loss: 0.367501	acc: 0.619687
loss: 0.352387	acc: 0.612500
loss: 0.563507	acc: 0.620938
loss: 0.341027	acc: 0.612969
loss: 0.380451	acc: 0.625625
loss

loss: 0.341781	acc: 0.609531
loss: 0.382459	acc: 0.590000
loss: 0.354487	acc: 0.596406
loss: 0.328344	acc: 0.599219
loss: 0.351721	acc: 0.605156
loss: 0.335061	acc: 0.611719
loss: 0.308629	acc: 0.595625
loss: 0.373489	acc: 0.609219
loss: 0.252103	acc: 0.610156
loss: 0.457800	acc: 0.614531
loss: 0.310702	acc: 0.604375
loss: 0.343924	acc: 0.603750
loss: 0.395718	acc: 0.597812
Epoch 11	Episode 33570	Cost Time 2.16	
loss: 0.357821	acc: 0.600781
loss: 0.323030	acc: 0.597656
loss: 0.308097	acc: 0.593281
loss: 0.408260	acc: 0.600938
loss: 0.349415	acc: 0.593594
loss: 0.308111	acc: 0.601875
loss: 0.292707	acc: 0.592656
loss: 0.364626	acc: 0.596094
loss: 0.377600	acc: 0.597344
loss: 0.284196	acc: 0.603437
loss: 0.213413	acc: 0.602344
loss: 0.399611	acc: 0.596094
loss: 0.349664	acc: 0.597031
loss: 0.327784	acc: 0.597187
loss: 0.388583	acc: 0.599375
loss: 0.355140	acc: 0.597187
loss: 0.362861	acc: 0.602656
loss: 0.312326	acc: 0.601719
loss: 0.352167	acc: 0.612500
loss: 0.355918	acc: 0.613437
loss

loss: 0.380481	acc: 0.599844
loss: 0.315084	acc: 0.620156
loss: 0.348830	acc: 0.606250
loss: 0.393523	acc: 0.615781
loss: 0.349370	acc: 0.618437
loss: 0.299406	acc: 0.622969
loss: 0.332624	acc: 0.612812
loss: 0.328884	acc: 0.607344
loss: 0.391265	acc: 0.608125
loss: 0.266425	acc: 0.615469
loss: 0.323259	acc: 0.600469
loss: 0.332592	acc: 0.605313
Epoch 11	Episode 33930	Cost Time 2.15	
loss: 0.301694	acc: 0.613437
loss: 0.350723	acc: 0.612500
loss: 0.374486	acc: 0.618750
loss: 0.343841	acc: 0.616719
loss: 0.394475	acc: 0.618437
loss: 0.341721	acc: 0.613437
loss: 0.271260	acc: 0.610469
loss: 0.325249	acc: 0.614062
loss: 0.238007	acc: 0.621563
loss: 0.353868	acc: 0.607500
loss: 0.413263	acc: 0.615938
loss: 0.252955	acc: 0.625625
loss: 0.369847	acc: 0.606250
loss: 0.349680	acc: 0.606406
loss: 0.351372	acc: 0.614062
loss: 0.300297	acc: 0.615469
loss: 0.255234	acc: 0.624219
loss: 0.313872	acc: 0.625625
loss: 0.300861	acc: 0.617969
loss: 0.371035	acc: 0.603281
loss: 0.382343	acc: 0.607656
loss

loss: 0.306929	acc: 0.614062
loss: 0.307403	acc: 0.625156
loss: 0.421762	acc: 0.612500
loss: 0.413697	acc: 0.616875
loss: 0.292022	acc: 0.617656
loss: 0.385295	acc: 0.613594
loss: 0.344398	acc: 0.620156
loss: 0.288481	acc: 0.605469
loss: 0.338210	acc: 0.632188
loss: 0.394624	acc: 0.615156
loss: 0.315822	acc: 0.625938
loss: 0.332698	acc: 0.621875
loss: 0.345483	acc: 0.613750
loss: 0.389916	acc: 0.604688
loss: 0.300281	acc: 0.629062
loss: 0.270420	acc: 0.621875
loss: 0.366745	acc: 0.621250
loss: 0.344237	acc: 0.621250
loss: 0.397645	acc: 0.615156
Epoch 11	Episode 34290	Cost Time 2.12	
loss: 0.279054	acc: 0.621406
loss: 0.417439	acc: 0.634531
loss: 0.299344	acc: 0.622344
loss: 0.326376	acc: 0.615000
loss: 0.401295	acc: 0.623437
loss: 0.407735	acc: 0.626719
loss: 0.358043	acc: 0.631875
loss: 0.328063	acc: 0.617812
loss: 0.351639	acc: 0.620938
loss: 0.380824	acc: 0.621406
loss: 0.357143	acc: 0.611094
loss: 0.334779	acc: 0.627344
loss: 0.303564	acc: 0.614531
loss: 0.387678	acc: 0.613750
loss

loss: 0.341668	acc: 0.632812
loss: 0.382127	acc: 0.622344
loss: 0.333583	acc: 0.623906
loss: 0.255541	acc: 0.630781
loss: 0.323748	acc: 0.631563
loss: 0.302682	acc: 0.625313
loss: 0.390554	acc: 0.614844
loss: 0.231472	acc: 0.635469
loss: 0.300451	acc: 0.624531
loss: 0.325975	acc: 0.623750
loss: 0.350214	acc: 0.628125
loss: 0.394706	acc: 0.616563
loss: 0.319270	acc: 0.622656
loss: 0.418062	acc: 0.626406
loss: 0.287959	acc: 0.625000
loss: 0.365273	acc: 0.626250
loss: 0.281651	acc: 0.627031
loss: 0.318032	acc: 0.626719
loss: 0.362003	acc: 0.615469
loss: 0.401788	acc: 0.624375
loss: 0.379284	acc: 0.622031
loss: 0.363227	acc: 0.629844
loss: 0.293205	acc: 0.621250
loss: 0.315474	acc: 0.630469
loss: 0.327679	acc: 0.625156
loss: 0.271158	acc: 0.623906
loss: 0.322662	acc: 0.629375
loss: 0.261612	acc: 0.627344
loss: 0.283667	acc: 0.620313
loss: 0.434719	acc: 0.616563
loss: 0.299580	acc: 0.634687
loss: 0.368257	acc: 0.623594
loss: 0.330162	acc: 0.616094
loss: 0.329649	acc: 0.626719
loss: 0.342361

In [ ]:
sl_agent

In [ ]:
submissionId = 16302889 # TeamVQ
allEpisodes = list_episodes_for_submission(submissionId)


episode = allEpisodes['result']['episodes'][-1]
replayId = episode['id']
agents = episode['agents']
agentsId = [d['submissionId'] for d in agents]
print(agentsId)
replay = get_episode_replay(replayId)
replay_dict = json.loads(replay['result']['replay'])
for i, v in enumerate(replay_dict['rewards']):
    if v == None:
        replay_dict['rewards'][i] = -2000
winnerId = agentsId.index(submissionId)
print(f"player: {winnerId},\treward: {replay_dict['rewards']}")
steps = len(replay_dict['steps'])

In [ ]:
step = 5
observation = replay_dict['steps'][step][0]['observation']
if winnerId != 0:
    observation['player'] = winnerId

my_action = replay_dict['steps'][step+1][winnerId]['action']
obs = Obs(observation)

state = obs.state
state1, state2 = state
action = actionToTensor(my_action, obs.shipPosDict, obs.shipyardPosDict)
for key, val in obs.shipPosDict.items():
    r, c = key
    c_state1 = generate_offset_map(state1, r, c)
    target = np.argmax(action[:6, r, c])
    break
    #print(f'ship state1: {c_state1}')
    #print(f'ship target: {target}')
    
"""for key, val in obs.shipyardPosDict.items():
    r, c = key
    c_state1 = generate_offset_map(state1, r, c)
    target = np.argmax(action[6:, r, c])"""

In [ ]:
action[:6, r, c]

In [ ]:
target

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
test = np.random.rand(1,5,5)

In [ ]:
test

In [ ]:
np.rot90(test, k=1, axes=(1,2))

In [ ]:
#state1, state2, action = sl_agent.memory.memory[55]

In [ ]:
#action

In [ ]:
state1 = np.reshape(c_state1, (1,9,21,21))
state2 = np.reshape(state2, (1,21))

In [ ]:
a = sl_agent.act(state1, state2, True)

In [ ]:
a

In [ ]:
list(zip(range(6), a[0]))

In [ ]:
fig = plt.figure(figsize=(12,12))

for i in range(1,10):
    plt.subplot(3,3,i)
    plt.imshow(state1[0,i-1])
    plt.colorbar()